In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline
from langdetect import detect
import  fuzzywuzzy.fuzz as fuzz
import time
from langdetect import detect

In [35]:
df=pd.read_csv("IndianFoodDatasetCSV.csv")
pd.set_option('display.max_rows', 500)

In [36]:
df.drop(["RecipeName","Ingredients","Instructions"],axis=1,inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

In [37]:
start=time.time()
df["lang"]=df["TranslatedIngredients"].apply(lambda x:detect(x))
end = time.time()
print(end - start)      

68.83786153793335


In [38]:
df=df[df["lang"]=="en"]

In [39]:
df["lang"].value_counts()

en    6276
Name: lang, dtype: int64

In [40]:
df.to_csv('df-lan.csv',index=False)

In [41]:
df=pd.read_csv("df-lan.csv")

In [42]:
def preprocess(text):
    text=re.sub(r'[0-9]+', '', text)
    delword="Teaspoon,teaspoon,teaspoons,-,/,to taste,as needed,shredded,cup,cups,washed,sliced,finely,few,chopped,tablespoons,\
    Fresh,optional,peeled ,Cubed,for garnishing,garnishing,grams,washed, finely chopped,chopped,frying,tablespoon,deseeded,thinly,\
    ,smearing,as per taste,as required, vegans can eliminate this ingredient, roasted,coarsely pounded,\
    ,required,plain,diced,minced,mixed with warm milk,boiled,steamed,as per taste,grated,made into paste,,cooked,\
    taste,(remove seeds fibre any),(Whole),peel cut into rings,(fine variety),cleaned,but recommended,(leftover water from washing rice)\
    to cook,quartered"
    
    stopwords = {'and','as','if','it','at','is','or',"gm","for","ml","whole"}
    
    for i in delword.split(","):
        text = text.replace(i, '')
    text = re.sub(r'\s+', ' ', text)
    text= re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', text)
    text=' '.join(filter(lambda x: x.lower() not in stopwords,  text.split()))
    return text

In [43]:
df1=df.copy()

In [44]:
df1["P-Ingredients"]=df1["TranslatedIngredients"].apply(preprocess)

In [45]:
df1["P-Ingredients"].values

array(['Karela (Bitter Gourd Pavakkai) ,Salt , Onion , Gram flour (besan), Turmeric powder (Haldi), Red Chilli powder, Cumin seeds (Jeera), Coriander Powder (Dhania), Amchur (Dry Mango Powder),Sunflower Oil',
       'rice , tomatoes, BC Belle Bhat powder, salt , chickpea lentils, cumin seeds, white urad dal, mustard, green chilli, dry red chilli, cashew peanuts, oilasafoetida',
       'Rice Vermicelli Noodles (Thin), Onion , Carrots (Gajjar) , Green peas (Matar), Green Chillies, Asafoetida (hing), Mustard seeds, White Urad Dal (Split), Ghee, sprig Curry leaves,Salt , Lemon juice',
       ...,
       'Quinoa, Sugar, Cardamom Powder (Elaichi), Milk vegans can substitute this ingredient with almond milk, Condensed Milk khoya (mawa), Ghee, Saffron strands,Mixed nuts',
       'Spring Onion (Bulb & Greens) , Tamarind Water, Gram flour (besan), Sunflower Oil, Mustard seeds, Cumin seeds (Jeera), Dry Red Chillies, pinch Asafoetida (hing), pinch Turmeric powder (Haldi),Salt to atste, Coriander (

In [46]:
import random
def rd():
    i=random.randint(0,6321)
    x=df1.loc[i,"P-Ingredients"]
    print(x)
    print(i)
    return x

In [47]:
tester=rd()

Chicken chicken gm, Baby corn baby corn gm, Onion , Green Bell Pepper (Capsicum) , Carrot (Gajjar) , Egg,Salt , Red Chilli flakes, Corn flour, White pepper powder, Schezwan sauce, Tomato Ketchup,Sunflower Oil to fry the chicken, Sugar, Button mushrooms , Asparagus , Carrot (Gajjar) , Onion ,Ginger ,Garlic , Fresh Red chillies, Sunflower Oil, Teriyaki Sauce,White pepper powder ,Salt , Fried Rice chicken ,French Fries
5570


In [48]:
df1["flag"]=0

In [49]:
user_ingredients='Gram flour, Curd , Green Chillies made into paste, Lemon juice, Enos fruit salt,Salt , Sunflower Oil, Mustard seeds, Cumin seeds (Jeera), Asafoetida (hing), Green Chillies, Red Chilli powder, Turmeric powder, Jaggery,Salt  , Sunflower Oil, Mustard seeds, Cumin seeds, Asafoetida , Green Chillies, Red Chilli powder, Turmeric, Jaggery , Tamarind ,Salt , Coriander '
len(user_ingredients.split(","))

26

In [50]:
df1.reset_index(drop=True,inplace=True)

In [51]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6276 entries, 0 to 6275
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Srno                    6276 non-null   int64 
 1   TranslatedRecipeName    6276 non-null   object
 2   TranslatedIngredients   6276 non-null   object
 3   PrepTimeInMins          6276 non-null   int64 
 4   CookTimeInMins          6276 non-null   int64 
 5   TotalTimeInMins         6276 non-null   int64 
 6   Servings                6276 non-null   int64 
 7   Cuisine                 6276 non-null   object
 8   Course                  6276 non-null   object
 9   Diet                    6276 non-null   object
 10  TranslatedInstructions  6276 non-null   object
 11  URL                     6276 non-null   object
 12  lang                    6276 non-null   object
 13  P-Ingredients           6276 non-null   object
 14  flag                    6276 non-null   int64 
dtypes: i

In [19]:
rows=6269

In [52]:
def Cook2(Pingredients):    
        "Comparing the Recipe Ingredients and User Ingredients and decreasing count value for each Recipe ingredient missing.\
        Count describes the no. of ingredients missing from user ingredients to make that recipe."
        flag,count=0,0
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j)>80:                    
                    flag=1 
                    break
            if (flag==0):
                count-=1
            else:
                flag=0
        return count

61.400166034698486


In [ ]:
df["flag"]=df["P-Ingredients"].apply(Cook2)

In [53]:
df1.loc[93,"P-Ingredients"]

'coconut milk, water, chana lentils, green chillies do more less you wish, oil, mustard, small Spoon turmeric powder, cumin seeds, curry leaves little, salt , coriander leaves little'

In [54]:
df1["flag"].value_counts()

4     654
0     532
6     449
12    429
11    424
8     395
9     394
13    379
10    373
5     352
14    306
7     284
15    260
3     256
16    175
2     171
1     161
17    112
18     84
19     43
20     19
21     13
22      8
24      1
26      1
23      1
Name: flag, dtype: int64

In [55]:
df1.sort_values(by="flag",ascending=False).head(10)

,Srno,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,URL,lang,P-Ingredients,flag
3293,4502,Rasawala Dhokla Recipe,"2 cups Gram flour (besan),1/2 cup Curd (Dahi /...",5,10,15,4,Gujarati Recipes﻿,North Indian Breakfast,Vegetarian,"To begin making the Rasawala Dhokla recipe, in...",http://www.archanaskitchen.com/rasawala-dhokla...,en,"Gram flour (besan), Curd (Dahi Yogurt), Green ...",26
4847,7665,Maharashtrian Golyachi Amti Recipe (Besan Ball...,"1 cup Gram flour (besan),1 tablespoon Sunflowe...",10,20,30,4,Maharashtrian Recipes,Side Dish,Vegetarian,To Make Balls/Golyachi (kofta)To begin Maharas...,http://www.archanaskitchen.com/maharashtrian-g...,en,"Gram flour (besan), Sunflower Oil, Ajwain (Car...",24
647,714,Dal Dhokli Recipe With Sprouted Moong Dal,"3/4 cup Whole Wheat Flour,1/4 cup Gram flour (...",10,45,55,4,Gujarati Recipes﻿,Lunch,Vegetarian,To begin making the Dal Dhokli Recipe With Spr...,https://www.archanaskitchen.com/dal-dhokli-rec...,en,"Wheat Flour, Gram flour (besan), Red Chilli po...",23
5386,9153,Tindora Ka Salan Recipe,250 grams Tindora (Dondakaya/ Kovakkai) - slit...,10,20,30,4,North Indian Recipes,Side Dish,Vegetarian,"To begin making the Tindora Ka Salan Recipe, d...",https://www.archanaskitchen.com/tindora-ka-sal...,en,Tindora (Dondakaya Kovakkai) slitted keeping t...,22
4304,6410,Andhra Style Mukkala Pulusu Recipe-Mixed Veget...,1/4 cup Mangalorean cucumber - cut into small ...,10,20,30,4,Andhra,Lunch,Vegetarian,"To begin making the Mukkala Pulusu Recipe, fir...",https://www.archanaskitchen.com/mukkala-pulusu...,en,"Mangalorean cucumber cut into small chunks, Ca...",22
302,323,Kurkuri Bhindi With Spicy Tomato Yogurt Gravy ...,"250 grams Bhindi (Lady Finger/Okra),Sunflower ...",20,40,60,4,North Indian Recipes,Lunch,Vegetarian,To begin making the Kurkuri Bhindi With Spicy ...,http://www.archanaskitchen.com/kurkuri-bhindi-...,en,"Bhindi (Lady FingerOkra),Sunflower Oil deep , ...",22
3815,5474,Aloo Bonda Recipe (Deep Fried Snack With Potat...,"5 Potatoes (Aloo) - boiled,2 teaspoon Mustard ...",10,45,55,2,Bengali Recipes,Snack,Vegetarian,"To begin making the Aloo Bonda recipe, firstly...",http://www.archanaskitchen.com/aloo-bonda-reci...,en,"Potatoes (Aloo) , Mustard seeds, Cumin seeds (...",22
6062,12021,Mangalorean Style Bendekayi Huli Koddel Recipe...,"2 cups Bhindi (Lady Finger/Okra),1 cup Yellow ...",10,30,40,4,Mangalorean,Lunch,Vegetarian,To begin making the Mangalorean Style Bendekay...,https://www.archanaskitchen.com/mangalorean-st...,en,"Bhindi (Lady FingerOkra), Yellow Moong Dal (Sp...",22
333,358,Potato Stuffed Dal Dhokli Recipe,"1 cup Whole Wheat Flour,Salt - to taste,1 tabl...",25,75,100,4,North Indian Recipes,Lunch,Vegetarian,"To prepare Potato Stuffed Dal Dhokli Recipe, g...",https://www.archanaskitchen.com/potato-stuffed...,en,"Wheat Flour,Salt , Sunflower Oil,Whole Wheat F...",22
3173,4285,Makai Dhokla Recipe - Savoury Steamed Corn Cake,"1 cup Sweet corn,1 cup Corn meal,1/2 cup Curd ...",20,45,65,4,Gujarati Recipes﻿,Snack,High Protein Vegetarian,"To begin making the Makai Dhokla, firstly crus...",http://www.archanaskitchen.com/makai-dhokla-sa...,en,"Sweet corn, Corn meal, Curd (Dahi Yogurt) whis...",22


In [ ]:
df1.reset_index(drop=True,inplace=True)

In [ ]:
df1.drop([5743,5173,4846,4125,3766,422,80,442,583,4994,5466,5463,5440,5383,5374,4994,1571,3311,3222,2305,1571,605,423],inplace=True)

In [ ]:
df.reset_index(drop=True,inplace=True)

In [56]:
df1.to_csv('df-en-processed.csv',index=False)

To Preprocess Instructions

In [2]:
df=pd.read_csv("df-en-processed.csv")

In [16]:
df.loc[5,"TranslatedInstructions"]

'To begin making\xa0Pudina Khara Pongal Recipe, wash and soak the rice and dal for 20 minutes.Make a paste of mint/pudina, coriander, green chilli and ginger using a mixer grinder and keep aside.Now, heat oil in the pressure cooker, add the cumin seeds and let it crackle.Add the soaked rice and dal and fry for a few minutes.Add the mint coriander paste and saute for a minute.Add 5 cups of water, season with salt and pressure cook for 4 to 5 whistles and keep aside.In the mean prepare the tadka by heating ghee in a small tadka pan.Add in the cumin seeds, peppercorns and let it crackle.Add the cashew halves and fry till they turn golden.Add the curry leaves and asafoetida and turn off the flame.Once the pressure has been released, open the lid and mash the pongal mixture.Add in the tadka over it and\xa0Serve the Pudina Khara Pongal Recipe along with\xa0Kathirikai Gothsu Recipe (Spicy and Tangy Eggplant Curry)and\xa0South Indian Coconut Chutney\xa0for a relaxed weekend breakfast.'

In [23]:
def pre(text):

    delword="\xa0,\n"    
    for i in delword.split(","):
        text = text.replace(i, ' ')
    return text

In [22]:
testtext='To begin making\xa0Pudina Khara Pongal Recipe, wash and soak the rice and dal for 20 minutes.Make a paste of mint/pudina, coriander, green chilli and ginger using a mixer grinder and keep aside.Now, heat oil in the pressure cooker, add the cumin seeds and let it crackle.Add the soaked rice and dal and fry for a few minutes.Add the mint coriander paste and saute for a minute.Add 5 cups of water, season with salt and pressure cook for 4 to 5 whistles and keep aside.In the mean prepare the tadka by heating ghee in a small tadka pan.Add in the cumin seeds, peppercorns and let it crackle.Add the cashew halves and fry till they turn golden.Add the curry leaves and asafoetida and turn off the flame.Once the pressure has been released, open the lid and mash the pongal mixture.Add in the tadka over it and\xa0Serve the Pudina Khara Pongal Recipe along with\xa0Kathirikai Gothsu Recipe (Spicy and Tangy Eggplant Curry)and\xa0South Indian Coconut Chutney\xa0for a relaxed weekend breakfast.'

In [24]:
d=pre(testtext)

In [25]:
d

'To begin making Pudina Khara Pongal Recipe, wash and soak the rice and dal for 20 minutes.Make a paste of mint/pudina, coriander, green chilli and ginger using a mixer grinder and keep aside.Now, heat oil in the pressure cooker, add the cumin seeds and let it crackle.Add the soaked rice and dal and fry for a few minutes.Add the mint coriander paste and saute for a minute.Add 5 cups of water, season with salt and pressure cook for 4 to 5 whistles and keep aside.In the mean prepare the tadka by heating ghee in a small tadka pan.Add in the cumin seeds, peppercorns and let it crackle.Add the cashew halves and fry till they turn golden.Add the curry leaves and asafoetida and turn off the flame.Once the pressure has been released, open the lid and mash the pongal mixture.Add in the tadka over it and Serve the Pudina Khara Pongal Recipe along with Kathirikai Gothsu Recipe (Spicy and Tangy Eggplant Curry)and South Indian Coconut Chutney for a relaxed weekend breakfast.'

In [26]:
df.columns

Index(['TranslatedRecipeName', 'TranslatedIngredients', 'PrepTimeInMins',
       'CookTimeInMins', 'TotalTimeInMins', 'Servings', 'Cuisine', 'Course',
       'Diet', 'TranslatedInstructions', 'URL', 'lang', 'P-Ingredients',
       'flag'],
      dtype='object')

In [27]:
df["instruc-lang"]=df["TranslatedInstructions"].apply(lambda x:detect(x))

In [28]:
df["instruc-lang"].value_counts()

en    5990
hi     265
Name: instruc-lang, dtype: int64

In [29]:
df=df[df["instruc-lang"]=="en"]

In [31]:
df["TranslatedInstructions"]=df["TranslatedInstructions"].apply(pre)

C:\Users\soory\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
df.loc[10,"TranslatedInstructions"]

'To begin making the homemade baked beans recipe, we first have to soak the beans overnight for about 8 hours in water.We also have to make the cooked tomato puree. Click here to see the recipe of how to make tomato puree at home.Once the beans are soaked well, our next step is to cook them. I like using the pressure cooker method to cook the beans. Place the beans in the pressure, place the weight on, cover the pan and cook it along with 2-1/2 cups of water. After you heat 4 to 5 whistles, turn the heat to low and cook for another 15 minutes.Turn off the heat and allow the pressure cooker to release its pressure naturally. This would take another 10 minutes. The beans continue to cook under the pressure even when it is turned off.Once the pressure is released, open the cooker and drain out the excess liquids. You store this liquid and use it in the vegetable stock or risotto.Once the beans are cooked, our next step is to spice it up to make the baked beans.Heat oil in a sauce pan on m

In [35]:
df.to_csv('df-en-processed.csv',index=False)

In [ ]:

df=pd.read_csv("df-en-processed.csv")
user_ingredients="Eggs hard & halved, Onion , Tomato , Coriander Powder (Dhania), Garam masala powder, Red Chilli powder, \
Cinnamon Stick (Dalchini), Cloves (Laung), Sunflower Oil,Salt , Coriander (Dhania) Seeds"


    
    return Cookdf

Cookdf=Cookman(df,user_ingredients)

In [ ]:
def Cookman(df,user_ingredients):
    def Cook2(Pingredients):    
        "Comparing the Recipe Ingredients and User Ingredients and decreasing count value for each Recipe ingredient missing.\
        Count describes the no. of ingredients missing from user ingredients to make that recipe."
        flag,count=0,0
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j)>80:                    
                    flag=1 
                    break
            if (flag==0):
                count-=1
            else:
                flag=0
        return count
    df["flag"]=df["P-Ingredients"].apply(Cook2)
    Cookdf=df.sort_values(by="flag",ascending=False).head(10)
    def Missing(Cookdf):
        flag=0
        miss=""
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j)>80:                    
                    flag=1 
                    break
            if (flag==0):
                miss+=i+","
            miss = miss[:-1]+"."
            else:
                flag=0
        return miss
    Cookdf["Missing"]=Cookdf["P-Ingredients"].apply(Missing)
    return Cookdf

In [46]:
miss=""
i=["hi","bi"]
for b in i:    
   miss+=b+","
miss = miss[:-1]+"."

In [47]:
miss

'hi,bi.'